# `GiRaFFE_NRPy`: Numerical Methods

## Authors: Patrick Nelson &

In this module, we will introduce the numerical methods with which we will solve the GRFFE equations.

`GiRaFFE_NRPy` supports uniform Cartesian grids at the moment. We plan to expand this using the many options NRPy+ offers us. The grids are, unless otherwise specified, cell-centered. We employ two different prescriptions; in the first, all quantities are sampled at cell centers, and in the second the vector potential, magnetic field, and scalar potential are staggered. They will be referred to as unstaggered and staggered, respectively.

Time evolution is accomplished through the method of lines (MOL) with a Runge-Kutta fourth-order scheme and finite-difference methods and an approximate Riemann solver for spatial derivatives.

Second-order finite-differencing methods are used in the computation of the magnetic field as the curl of the vector potential, as well as in the gauge term of the vector potential evolution equation, $\partial_i (\alpha \Phi - \beta^j A_j)$, and the evolution equation for the scalar potential $\Phi$. 

### Step 0: Initial Data

Initial data is computed by the `GiRaFFEfood_NRPy` modules and sets the vector potential $A_i$ and Valencia three-velocity $\bar{v}^i$. The scalar potential $\Phi$ is always set to 0. 

The physical scenarios for which we generate initial data are presented as expressions for the vector potential $A_i$ and electric field $E_i$. After setting the vector potential directly, the velocity is calculated as $$\bar{v}^i = \frac{\epsilon^{ijk} E_j B_k}{B^2}$$, where $\epsilon^{ijk} = [ijk]/\sqrt{\gamma}$, $\gamma$ is the three-metric determinant, and $B_k = \gamma_{ik} B^i$ is calculated analytically as $B^i = \epsilon^{ijk} \partial_j A_k$.

We then numerically set $B^i$ according to the same formula as above, but using second-order finite-differencing. Finally, the densitized Poynting flux is set numerically as $\bar{v}^i = 4 \pi \frac{\gamma^{ij} {\tilde S}_j}{\sqrt{\gamma} B^2}$.

### Step 1a: Evolution equations--unstaggered

In the unstaggered prescription, the gauge terms are calculated first: the gauge term of the vector potential evolution equation, $\partial_i (\alpha \Phi - \beta^j A_j)$, and the evolution equation for the scalar potential $\Phi$. This is done using second-order finite differencing.

However, for the flux terms in the evolution equations for the vector potential and densitized Poynting flux, we use something a bit more complicated. The methods we use here involve computing the terms on cell faces, and either differencing (for $\tilde{S}_i$) or averaging (for $A_i$) the values on opposing faces. To do this, we first interpolate the metric quantities to cell faces in direction $i$ with basic, third-order-accurate polynomial interpolation. Then, we reconstruct the primitive variables on the cell faces in the same direction using the piecewise-parabolic method of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf). The $\tilde{S}_i$ source term $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$ is computed, using the interpolated grid function values to compute the metric derivatives $\partial_i g_{\mu \nu}$.

Then, we solve the one-dimensional Riemann problem approximately on the cell faces according to the method of [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002) and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021) (hereafter HLLE). 

### Step 1b: Evolution equations--staggered

In the staggered prescription, the steps are done in a slightly different order. Because $\Phi$ is sampled at cell vertices and $A_i$ is sampled at cell edges, updating the gauge terms will require interpolated gridfunctions and reconstructed primitives; as such, it is performed after the flux terms are calculated.

The right-hand side of the vector potential $A_i$ is calculated using a two-dimensional approximate Riemann solver described by [Del Zanna, Bucciantini and Londrillo](https://arxiv.org/abs/astro-ph/0210618).

### Step 1c: Boundary conditions--Vector Potential

We do not have exact boundary conditions available for the problems we would like to solve, so we choose simple boundary conditions to preserve numerical stability. For the vector potential, weuse simple linear extrapolation boundary conditions.

### Step 1d: Primitive recovery

Now that the vector potential has been updated everywhere, the magnetic field is computed on the whole grid using the same function that was used during the initial data step. 

We enforce constraints on the densitized Poynting flux; we force it to be strictly orthogonal to the magnetic field and guarantee that the speed of light is not violated (note that $\tilde{S}_i$ is very closely related to $\bar{v}^i$). We than calculate the Valencia three-velocity $\bar{v}^i$. Note that this step is far simpler than in full ideal GRMHD because $\tilde{S}_i\left(\bar{v}^i\right)$ can be inverted analytically.

We then apply an algorithm to the newly-updated velocities to preserve any current sheets that form perpendicular to the $z$-axis and recalculate $\tilde{S}_i$ using the same method we used during the initial data step.

### Step 1e: Boundary conditions--three-velocity

Now that the Valencia three-velocity has been updated, we apply copy boundary conditions.

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-Numerical_Methods")